## Scraping Map Data 
- source url : http://www.housinginnovationalliance.com/whats-new/off-site-heat-map
- get company name, address, decs, website, email, label

## Page load with lib
- get the page with `request`
- Use `Bs4` for webpage Process
- Collect the info
- Create the func
- Create `CSV` with that info

## library

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Request func
- `get_map_data` for maps data with json format
- `get_company_details` for company data with json format

In [ ]:
# def get_legends():
#     headers = {
#         'Connection': 'keep-alive',
#         'Accept': '*/*',
#         'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36',
#         'X-Requested-With': 'XMLHttpRequest',
#         'Referer': 'http://www.zeemaps.com/pub?group=3839111&legend=1&search=1&simpleadd=1&track=UA-41067143-4&x=-100.193057&y=43.712520&z=13',
#         'Accept-Language': 'en-US,en;q=0.9',
#     }
#     params = (
#         ('g', '3839111'),
#     )
#     response = requests.get('http://www.zeemaps.com/legends/getall', headers=headers, params=params, verify=False)
#     return response.json()

def get_map_data():
    headers = {
        'Connection': 'keep-alive',
        'Accept': '*/*',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36',
        'X-Requested-With': 'XMLHttpRequest',
        'Referer': 'http://www.zeemaps.com/pub?group=3839111&legend=1&search=1&simpleadd=1&track=UA-41067143-4&x=-100.193057&y=43.712520&z=13',
        'Accept-Language': 'en-US,en;q=0.9',
    }

    params = (
        ('g', '3839111'),
        ('k', 'REGULAR'),
        ('e', 'true'),
        ('_dc', '0.9372072014948007'),
    )

    response = requests.get('http://www.zeemaps.com/emarkers', headers=headers, params=params, verify=False)
    return response.json()

def get_company_details(c_id):
    headers = {
        'Connection': 'keep-alive',
        'Accept': 'application/json, text/javascript, */*; q=0.01',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36',
        'X-Requested-With': 'XMLHttpRequest',
        'Referer': 'http://www.zeemaps.com/pub?group=3839111&legend=1&search=1&simpleadd=1&track=UA-41067143-4&x=-100.193057&y=43.712520&z=13',
        'Accept-Language': 'en-US,en;q=0.9',
    }

    params = (
        ('g', ['3839111', '3839111']),
        ('j', '1'),
        ('sh', ''),
        ('_dc', '0.2890865354260024'),
        ('eids', f'[{c_id}]'),
        ('emb', '1'),
    )

    response = requests.get('http://www.zeemaps.com/etext', headers=headers, params=params, verify=False)
    return response.json()

In [ ]:
len(get_map_data())

991

## company list
- get the company details from each `eid` of map data 

In [ ]:
company_details_list = []

for item in  get_map_data()[0:20]:
    company_id = item['id']
    company_details = get_company_details(company_id)
    company_details_list.append(company_details)

In [ ]:
company_details_list[4]

{'a': '',
 'ad': {'city': 'New Bedford',
  'country': 'United States',
  'postcode': '01440',
  'state': 'MA',
  'street': '200 Welby Road'},
 'eid': '268449605',
 'h': 'New Bedford, MA 01440',
 'i': '',
 'l': False,
 'lat': 41.710064,
 'lng': -70.951811,
 'pinnedCircle': 0,
 't': "<center class='markersubtitle'><a target='_blank' href='http://www.reliabletruss.com'>Website</a></center><span class='bold'>Description</span>&nbsp;Since 1954, New England builders have trusted Reliable Truss, a division of National Lumber, for all their structural elements. We are experts in wood trusses, structural components, and materials. Reliable Truss is also a leading distributor of Boise Cascade wood I-Joists and beams. For both residential and commercial projects, we design and manufacture wood roof trusses, wall panels, timber trusses, and floor panels.<br/><span class='bold'>Phone</span>&nbsp;<span class='phone'>508-998-7877</span><br/><span class='bold'>Services</span>&nbsp;Design, Mfr<br/><spa

## scrape the `t` key
- collect all items from `t`
- store in list as a dict

In [ ]:
html_list = []

for company in company_details_list:
    html_doc = company['t']
    soup = BeautifulSoup(html_doc, 'html.parser')
    company_data_dict = {}
    try:
        company_data_dict['website'] = soup.find('a')['href']
    except:
        company_data_dict['website'] = ''
    try:
        company_data_dict['phone'] = soup.find('span',{'class':"phone"}).text
    except:
        company_data_dict['phone'] = ''    
    for item in soup.find_all('span'):
        if '<br/>' not in str(item.next_sibling) and 'Phone' not in item.text :
            company_data_dict[item.text] = str(item.next_sibling).replace('\xa0','')
    html_list.append(company_data_dict)


In [ ]:
len(company_details_list)


20

In [ ]:
len(html_list)

20

## Save data
- remove unnecessary items
- save as `df/csv` 

In [ ]:
main_list = []
for company, html in zip(company_details_list, html_list):
    all_data_dict = {**company, **html}
    all_data_dict = {**all_data_dict, **company['ad']}
    del all_data_dict['ad']
    del all_data_dict['t']
    main_list.append(all_data_dict)

In [ ]:
pd.DataFrame(main_list)

,a,eid,lng,twts,h,i,pinnedCircle,title,l,lat,website,phone,Description,Services,Primary Structural Material,country,city,street,postcode,state,Affiliation,Email,Production Housing Types
0,,268449601,-72.425715,,"Belchertown, MA 01007",,0,UFP - Belchertown LLC,False,42.297956,https://www.ufpi.com/belchertown,(413) 323-7247,UFP Belchertown is a leading Northeastern supp...,Mfr,Wood,United States,Belchertown,155 Bay Road,01007,MA,NaN,NaN,NaN
1,,268449602,-72.723432,,"Westfield, MA 01085",,0,"Toll Integrated Systems, Inc.",False,42.156644,https://www.tisbuildingsolutions.com/,413-562-3861,"Toll Integrated Systems (TIS), a Toll Brothers...","Design, Mfr",Wood,United States,Westfield,100 Apremont Way,01085,MA,SBCA,NaN,NaN
2,,268449603,-72.723432,,"Westfield, MA 01085",,0,"Toll Integrated Systems, Inc.",False,42.156644,https://www.tisbuildingsolutions.com/,413-562-3861,"Toll Integrated Systems (TIS), a Toll Brothers...",NaN,NaN,United States,Westfield,100 Apremont Way,01085,MA,SBCA,,NaN
3,,268449604,-71.954649,,"Gardner, MA 01440",,0,Top Notch Truss,False,42.564814,,978-630-3916,NaN,NaN,NaN,United States,Gardner,101 Linus Allain Ave,01440,MA,SBCA,NaN,NaN
4,,268449605,-70.951811,,"New Bedford, MA 01440",,0,"Reliable Truss & Components, Inc.",False,41.710064,http://www.reliabletruss.com,508-998-7877,"Since 1954, New England builders have trusted ...","Design, Mfr",Wood,United States,New Bedford,200 Welby Road,01440,MA,NaN,NaN,NaN
5,,268449606,-72.305929,,"Keene, NH 03431",,0,Tektoniks (Bensonwood + Unity),False,42.919801,https://tektoniks.com/,603-756-3600,"We build insulated wall systems, insulated roo...","Design, Mfr",Wood,United States,Keene,25 Production Ave,03431,NH,NaN,NaN,"Single Family, Multifamily"
6,,268449607,-72.489070,,"West Chesterfield, NH 03466",,0,Foard Panels,False,42.892983,https://www.foardpanel.com/,"Pete Dunnigan, 603-256-8800","We have been working with SIPs since 1985 and,...","Design, Mfr, Install",SIPS,United States,West Chesterfield,53 Stow Drive,03466,NH,SIPA,,"Single Family, Multifamily"
7,,268449608,-70.447763,,"Saco, ME 04072",,0,Boise Cascade,False,43.530578,https://www.bc.com/,207-284-1600,roof and floor trusses,Mfr,Wood,United States,Saco,68 Industrial Park Rd,04072,ME,SBCA,NaN,NaN
8,,268449609,-68.045092,,"Presque Isle, ME 04769",,0,Aroostook Trusses Inc.,False,46.702675,http://www.aroostooktrusses.com,207-768-5817,"We manufacture engineered wood floor trusses, ...","Design, Mfr, Install",Wood,United States,Presque Isle,655 Missile Street,04769,ME,NaN,NaN,NaN
9,,268449610,-69.192930,,"Searsmont, ME 04973",,0,Ecocor,False,44.362230,https://www.ecocor.us/rethink-building,,Ecocor High Performance Buildings is a Searsmo...,"Design, Mfr, Install",Wood,United States,Searsmont,22 Main St. Bld #6,04973,ME,NaN,NaN,Single Family
